In [ ]:
! pip install yt-dlp
! pip install torchvideo

import torch
import nltk
nltk.download('punkt')

model = torch.hub.load('verlab/TextDrivenVideoAcceleration_TPAMI_2022:main', 'TextDrivenVideoAcceleration', pretrained=True, progress=True)
model.cuda()
model.eval()

In [ ]:
VIDEO_ID = 'jnewhlK2USg'
document = ['chop some tuna meat into small pieces',
            'mix the tuna pieces and some fish eggs in a bowl',
            'add hot sauce and mayo and then mix them together',
            'spread some cooked rice on seaweed sushi wrapper',
            'add the tuna mix on top and season with some sesame seeds',
            'roll it up and press firmly',
            'cut the roll into small rolls']

! yt-dlp https://www.youtube.com/watch?v=$VIDEO_ID -o $VIDEO_ID --merge-output-format mp4 # Download Video File

print('\nExtracting VDAN+ Features...')
semantic_embeddings = model.get_semantic_embeddings(video_filename=f'{VIDEO_ID}.mp4', document=document)

print('\nFast Forwarding Video...')
sf = model.fast_forward_video(video_filename=f'{VIDEO_ID}.mp4',
                              document=document,
                              desired_speedup=12,
                              output_video_filename=f'{VIDEO_ID}_FF.avi',
                              semantic_embeddings=semantic_embeddings)

In [ ]:
! wget -O youcookii_annotations_trainval.tar.gz http://youcook2.eecs.umich.edu/static/YouCookII/youcookii_annotations_trainval.tar.gz
! tar -xf youcookii_annotations_trainval.tar.gz
! rm youcookii_annotations_trainval.tar.gz

import json
import cv2
import numpy as np
from matplotlib import pyplot as plt

annotations = json.load(open('youcookii_annotations_trainval.json'))

video_annotations = annotations['database'][VIDEO_ID]['annotations']
cmap = plt.cm.get_cmap('hsv', len(video_annotations) + 1)
video_fps = cv2.VideoCapture(f'{VIDEO_ID}.mp4').get(5)
for idx, region in enumerate(video_annotations):
    video_region = region['segment']
    plt.axvspan(round(video_region[0] * video_fps), round(video_region[1] * video_fps), alpha=0.5, color=cmap(idx))

plt.scatter(sf[:-1], np.array(sf[1:])-np.array(sf[:-1]))
plt.title('Agent Skips')
plt.xlabel('Frame Number')
plt.ylabel('Skip')
plt.show()

In [ ]:
from google.colab import files
files.download(f'{VIDEO_ID}_FF.avi')